In [4]:
from census_area import Census
import json
import config as cfg

Set up the Census library's query object.

**Replace with an actual API key, but do not push it to the public repo.**

In [30]:
c = Census(cfg.api_key, year=2014)

Read in sample data--we are using a water district boundary for test purposes to figure out the aggregation logic.

In [ ]:
path = 'data_input/'
fname = 'water_district_boundary_sample.geojson'
with open(path+fname) as infile:
    my_shape_geojson = json.load(infile)

Build json from the [generator](https://wiki.python.org/moin/Generators) the .geo_tract() method returns

This json will contain all the census tracts within the water district boundary.

It will also contain the data for the Census tables we are quering for--in this case the number of homes that were built before 1939 (B25034_010E) as well as that statistic's margin or error (B25034_010M).

In [29]:
features = []
old_homes = c.acs5.geo_tract(('NAME', 'B25034_010M', 'B25034_010E'), my_shape_geojson['features'][0]['geometry'])
for tract_geojson, tract_data in old_homes:
    tract_geojson['properties'].update(tract_data)
    features.append(tract_data)

my_shape_with_new_data_geojson = {'type': "FeatureCollection", 'features': features}
my_shape_with_new_data_geojson

{'features': [{u'B25034_010E': 0.0,
   u'B25034_010M': 17.0,
   u'NAME': 'Census Tract 320.27, Orange County, California',
   u'county': '059',
   u'state': '06',
   u'tract': '032027'},
  {u'B25034_010E': 0.0,
   u'B25034_010M': 17.0,
   u'NAME': 'Census Tract 320.22, Orange County, California',
   u'county': '059',
   u'state': '06',
   u'tract': '032022'},
  {u'B25034_010E': 0.0,
   u'B25034_010M': 17.0,
   u'NAME': 'Census Tract 320.13, Orange County, California',
   u'county': '059',
   u'state': '06',
   u'tract': '032013'},
  {u'B25034_010E': 0.0,
   u'B25034_010M': 12.0,
   u'NAME': 'Census Tract 320.12, Orange County, California',
   u'county': '059',
   u'state': '06',
   u'tract': '032012'},
  {u'B25034_010E': 24.0,
   u'B25034_010M': 28.0,
   u'NAME': 'Census Tract 626.47, Orange County, California',
   u'county': '059',
   u'state': '06',
   u'tract': '062647'},
  {u'B25034_010E': 39.0,
   u'B25034_010M': 32.0,
   u'NAME': 'Census Tract 626.41, Orange County, California',


A simple statistic to aggregate is a count. This could be a population or, as in this case, the number of old homes.

Below is the logic to get the total sum of old homes within this water district.

In [26]:
old_homes_count = 0
for feature in my_shape_with_new_data_geojson['features']:
    old_homes_count += feature['B25034_010E']

old_homes_count

472.0

How would the logic change if we wanted to aggregate the margin of error?

Hint: [See here](https://www.census.gov/content/dam/Census/library/publications/2018/acs/acs_general_handbook_2018_ch08.pdf)

How would the logic change if we wanted to aggregate an average?

Hint: if we were trying to aggregate an average income, we would not only need to pull down a value for each census tract, but also each census tract's population to weight each value appropriately.